# Activity Recognition with Healthy Older People Using a Batteryless Wearable Sensor

This notebook provides code for loading the [activity recognition dataset](https://archive.ics.uci.edu/dataset/427/activity+recognition+with+healthy+older+people+using+a+batteryless+wearable+sensor) into a suitable format for classification and sequence labelling. 

The code is divided into three sections:
* Loading the data
* Sequence labelling: processing the data into a suitable format for sequence labelling. 
* Classification: processing the data into a suitable format for use with an IID classifier.

We recommend running all three sections, then using the variables produced by the code as required for sequence labelling and IID classification.


## Loading the Data

Run the following cells to load the data from disk. You will need to run code from the subsequent sections (or your own code) to get the data into a suitable format for classification and sequence labelling.  

In [4]:
import os

data_path = "activity_recognition_dataset/S1_Dataset"

files = os.listdir(data_path)
print(files)

['d1p01M', 'd1p02M', 'd1p03M', 'd1p04M', 'd1p05M', 'd1p06M', 'd1p07M', 'd1p08F', 'd1p09F', 'd1p10F', 'd1p11F', 'd1p12F', 'd1p13F', 'd1p14F', 'd1p15F', 'd1p16F', 'd1p17F', 'd1p18F', 'd1p19F', 'd1p20F', 'd1p21F', 'd1p22F', 'd1p23F', 'd1p24F', 'd1p25F', 'd1p26F', 'd1p27F', 'd1p28F', 'd1p29F', 'd1p30F', 'd1p31F', 'd1p32F', 'd1p33F', 'd1p34F', 'd1p35F', 'd1p36M', 'd1p37M', 'd1p38M', 'd1p39M', 'd1p40M', 'd1p41M', 'd1p42M', 'd1p43M', 'd1p44M', 'd1p45M', 'd1p46M', 'd1p47M', 'd1p48M', 'd1p49F', 'd1p50F', 'd1p51F', 'd1p52F', 'd1p53F', 'd1p54F', 'd1p55F', 'd1p56F', 'd1p57F', 'd1p58F', 'd1p59F', 'd1p60F', 'README.txt']


In [5]:
import pandas as pd  # use pandas to load data from CSV files

combined_data = []

for file in files:
    
    if file == "README.txt":
        continue
    
    try:
        # load a single sequence from the file
        seq_dataframe = pd.read_csv(os.path.join(data_path, file), header=None, names=["time", "frontal acc", "vertical acc", "lateral acc", "antenna ID", "RSSI", "phase", "frequency", "label"])
        
        # put the ID of the sequence into the dataframe as an extra column
        seq_dataframe['seqID'] = file  # use filename as ID
                
        combined_data.append(seq_dataframe)  # put the data frame into a list
    except:
        print(f"Could not load file {file}. Skipping.")

In [6]:
import numpy as np
import matplotlib.pyplot as plt

from hmmlearn import hmm
from IPython import display

import nltk
from nltk.corpus import brown
from nltk.probability import ConditionalFreqDist,ConditionalProbDist,MLEProbDist
import itertools

from sklearn.model_selection import train_test_split


# interactive display
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'

In [7]:
print(combined_data)

[       time  frontal acc  vertical acc  lateral acc  antenna ID  RSSI  \
0      0.00      0.27203       1.00820    -0.082102           1 -63.5   
1      0.50      0.27203       1.00820    -0.082102           1 -63.0   
2      1.50      0.44791       0.91636    -0.013684           1 -63.5   
3      1.75      0.44791       0.91636    -0.013684           1 -63.0   
4      2.50      0.34238       0.96229    -0.059296           1 -63.5   
..      ...          ...           ...          ...         ...   ...   
396  251.00      0.87003       0.46851     0.009122           4 -58.5   
397  251.25      0.87003       0.46851     0.009122           1 -57.5   
398  251.50      0.87003       0.46851     0.009122           1 -59.0   
399  253.25     -0.00938       0.95081    -0.116310           1 -58.5   
400  254.00     -0.00938       0.95081    -0.116310           1 -56.5   

       phase  frequency  label   seqID  
0    2.42520     924.25      1  d1p01M  
1    4.73690     921.75      1  d1p01M  

## Sequence Labelling

The following cell creates two variables that you can use for sequence labelling:
* X_by_seq is a list of 2-D numpy arrays. Each numpy array in this list corresponds to on data sequence and contains the input feature values for that sequence.
* y_by_seq is a list of 1-D numpy arrays, where each array contains the target class labels for each sequence. 

In [8]:
import numpy as np

# the pandas dataframe stores data in a table with headers
input_cols = ["frontal acc", "vertical acc", "lateral acc", "RSSI", "phase", "frequency"]  # column headers for the input features
output_col = "label"  # column header for the output label

# get the relevant columns from the pandas dataframes and convert to numpy arrays
X_by_seq = []  # store a list of numpy arrays containing the input features for each sequence
y_by_seq = []  # store a list of 1-D numpy arrays containing the target activity labels for each sequence

for seq_table in combined_data:
    X_by_seq.append(seq_table[input_cols].values)
    y_by_seq.append(seq_table[output_col].values - 1)    # subtract one from the label so that labels start from 0

n_states = np.unique(np.concatenate(y_by_seq)).size  # how many classes/states are there?

In [9]:
print(X_by_seq[0].shape)
print(y_by_seq[0].shape)
print(n_states)
print(X_by_seq[0])
print(y_by_seq[0])

(401, 6)
(401,)
4
[[ 2.7203e-01  1.0082e+00 -8.2102e-02 -6.3500e+01  2.4252e+00  9.2425e+02]
 [ 2.7203e-01  1.0082e+00 -8.2102e-02 -6.3000e+01  4.7369e+00  9.2175e+02]
 [ 4.4791e-01  9.1636e-01 -1.3684e-02 -6.3500e+01  3.0311e+00  9.2375e+02]
 ...
 [ 8.7003e-01  4.6851e-01  9.1224e-03 -5.9000e+01  2.5157e-01  9.2475e+02]
 [-9.3803e-03  9.5081e-01 -1.1631e-01 -5.8500e+01  3.5282e-01  9.2425e+02]
 [-9.3803e-03  9.5081e-01 -1.1631e-01 -5.6500e+01  5.1388e+00  9.2325e+02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3
 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 

### Variables for sequence labelling

The cell below produces the data you need for sequence labelling... You should be able to work with these variables directly.

In [10]:
from sklearn.model_selection import train_test_split

# create train/test split. Sequences are kept complete.
X_by_seq_tr, X_by_seq_test, y_by_seq_tr, y_by_seq_test = train_test_split(X_by_seq, y_by_seq, test_size=0.2, random_state=21)  # You may wish to make further splits of the data or to modify this split.

print(X_by_seq_tr[0].shape)
print(X_by_seq_test[0].shape)
print(y_by_seq_tr[0].shape)
print(y_by_seq_test[0].shape)

(293, 6)
(103, 6)
(293,)
(103,)


You may find the following code useful for creating a sequence labelling model:

In [11]:
# Record which observations occur given each state (activity label)
X_given_y = [[] for _ in range(n_states)]  # empty list where we will record the observations that occur given each activity label

for s, X_seq in enumerate(X_by_seq_tr):
    for i in range(X_seq.shape[0]):
        state_i = y_by_seq_tr[s][i]
        # print(state_i, X_seq[i, :][None, :])
        X_given_y[state_i].append(X_seq[i, :][None, :])

In [12]:
# print(X_given_y)
print(X_given_y[0][0])
print(X_given_y[1][0])
print(X_given_y[2][0])
print(X_given_y[3][0])
# print(X_given_y[4][0])
print("_______________")
print(X_given_y[0][1])
print(X_given_y[0][2])
print(X_given_y[0][3])
print(X_given_y[0][4])

# for j in range(6):
#     for i in range(n_states):
#         print(i,j, X_given_y[i][j])

# data = [(label, observations) for (label, observations) in enumerate(X_given_y)]
# data[0:5]
# labels = {label for label in X_given_y}
# print(f'Number of possible labels: {len(labels)}')
# print(f'Possible labels: {len(labels)}')

[[ 2.3685e-01  1.0082e+00  3.1928e-02 -5.5000e+01  1.5033e+00  9.2475e+02]]
[[-3.2597e-01  9.2784e-01  5.4735e-02 -5.7500e+01  8.1454e-01  9.2325e+02]]
[[ 1.6650e-01 -1.1713e-01  1.0468e+00 -5.6000e+01  4.1049e+00  9.2525e+02]]
[[ 3.8928e-01  9.5081e-01  7.7541e-02 -5.6000e+01  3.7874e+00  9.2225e+02]]
_______________
[[ 2.3685e-01  1.0082e+00  3.1928e-02 -5.6500e+01  3.5680e+00  9.2475e+02]]
[[ 2.3685e-01  1.0082e+00  3.1928e-02 -5.8000e+01  4.8550e+00  9.2075e+02]]
[[ 2.3685e-01  1.0082e+00  3.1928e-02 -5.6500e+01  3.9653e+00  9.2425e+02]]
[[ 2.3685e-01  1.0082e+00  3.1928e-02 -5.9500e+01  2.9452e-01  9.2025e+02]]


In [13]:
# print(X_by_seq_tr[0])
# print(y_by_seq_tr[0])
# train_labelled_activity = [ tup for sent in X_by_seq_tr for tup in sent ]
# test_labelled_activity = [ tup for sent in X_by_seq_test for tup in sent ]
# print(f'Number of labelled_activity in the training set: {len(train_labelled_activity)}')
# print(f'Number of labelled_activity in the test set: {len(test_labelled_activity)}')

In [14]:
from hmmlearn import hmm  # We recommend using this class if builing an HMM

# Record the mean feature values for observations in each state
means = np.zeros((n_states, len(input_cols)))

# Record the variance of feature values for observations in each state
diagonal_covars = np.zeros((n_states, len(input_cols)))

for state in range(n_states):
    means[state] = np.mean(X_given_y[state], axis=0)
    diagonal_covars[state, :] = np.var(X_given_y[state], axis=0)

In [15]:
A = np.array([
    [1, 2, 3, 4, 5, 6],
    [1, 2, 3, 4, 5, 6],
    [1, 2, 3, 4, 5, 6],
    [1, 2, 3, 4, 5, 6],])

print(A.shape)
print(diagonal_covars)
print(means)

(4, 6)
[[4.69970016e-02 9.98470840e-03 1.28309449e-02 8.06231607e+00
  5.02954464e+00 2.84021944e+00]
 [5.96488122e-02 9.72872064e-03 1.15226042e-02 1.47999385e+01
  4.94215122e+00 3.25274860e+00]
 [2.73339239e-02 1.56904378e-02 3.95137602e-02 2.62630241e+01
  4.99087067e+00 2.82984972e+00]
 [4.42563563e-02 1.43644094e-02 1.99985712e-02 5.51123430e+01
  4.64365093e+00 2.81114675e+00]]
[[ 3.57468029e-01  9.33799900e-01  5.73068933e-02 -5.89921417e+01
   3.15720544e+00  9.22742689e+02]
 [ 5.51372453e-01  8.66495244e-01  6.65145311e-02 -5.81649136e+01
   3.48891218e+00  9.22653706e+02]
 [ 1.09237509e+00  8.96515496e-04 -2.67407824e-02 -5.84239865e+01
   3.32236377e+00  9.22799698e+02]
 [ 2.23426814e-01  9.65194724e-01  2.29370871e-02 -5.41901012e+01
   3.14808656e+00  9.22885264e+02]]


In [16]:
transmat = np.zeros((n_states, len(input_cols)))
print(transmat.shape)
print(means.shape)

(4, 6)
(4, 6)


In [17]:
X_train = np.concatenate(X_by_seq_tr)
print(X_train.shape)
lengths_X_train = [len(x) for x in X_by_seq_tr]

X_test = np.concatenate(X_by_seq_test)
print(X_test.shape)
lengths_X_test = [len(x) for x in X_by_seq_test]

y_train = np.concatenate(y_by_seq_tr)
print(y_train.shape)
lengths_y_train = [len(x) for x in y_by_seq_tr]

y_test = np.concatenate(y_by_seq_test)
print(y_test.shape)
lengths_y_test = [len(x) for x in y_by_seq_test]

(46509, 6)
(5973, 6)
(46509,)
(5973,)


In [18]:
model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag").fit(X_train, lengths_X_train)
# model.means_ = means
# model.covars_ = diagonal_covars
# Predict the most likely sequence of hidden states for a new sequence
hidden_states = model.predict(X_test, lengths_X_test)

# Now hidden_states contains the predicted hidden states for the test sequence
print("Predicted hidden states for the test sequence:")
print(hidden_states, hidden_states.shape)

# with np.printoptions(threshold=np.inf):
#     print(hidden_states)

c:\Users\Ram\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Ram\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Predicted hidden states for the test sequence:
[1 1 1 ... 1 1 1] (5973,)


In [22]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, hidden_states)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.14


In [21]:
# Get the transition matrix from the trained HMM model
transition_matrix = model.transmat_

        
# Plot the transition matrix
fig, ax = plt.figure(figsize=(8, 6))

for i in range(4):
    for j in range(4):
        if i == j:
            text = ax.text(j, i, round(transition_matrix[i, j], 4),
                       ha="center", va="center", color="black")
        else:
            text = ax.text(j, i, round(transition_matrix[i, j], 4),
                       ha="center", va="center", color="w")

plt.imshow(transition_matrix, cmap='viridis')
plt.colorbar(label='Transition Probability')
plt.title('HMM Transition Matrix')
plt.show()


TypeError: cannot unpack non-iterable Figure object

<Figure size 800x600 with 0 Axes>

In [ ]:
# Assuming model is your trained HMM Gaussian model

# Get the means of the emission distributions for each hidden state
means = model.means_

print(model.means_)

# Calculate differences in means between hidden states for each feature
for i in range(means.shape[1]):  # Loop through features
    feature_means = means[:, i]
    mean_diff = feature_means.max() - feature_means.min()
    print(f"Feature {i+1},  Mean Difference: {mean_diff}")


[[ 3.22310152e-01  9.53350899e-01  4.23918530e-02 -5.82397180e+01
   3.21429059e+00  9.22736894e+02]
 [ 1.08966990e+00  7.25099924e-02 -1.02337708e-01 -5.88264491e+01
   3.14034579e+00  9.22871414e+02]
 [ 7.30978533e-01  5.11399295e-01  3.02376712e-01 -5.90241365e+01
   3.27150277e+00  9.22956343e+02]
 [ 1.14105830e+00 -8.92862782e-02 -2.18784381e-02 -5.78602676e+01
   3.54939660e+00  9.22642976e+02]]
Feature 1,  Mean Difference: 0.8187481512608409
Feature 2,  Mean Difference: 1.042637177046755
Feature 3,  Mean Difference: 0.40471442010329844
Feature 4,  Mean Difference: 1.163868912020611
Feature 5,  Mean Difference: 0.40905080875192246
Feature 6,  Mean Difference: 0.3133664746112572


## IID Classification

In this code, we take the X and y lists produced for sequence labelling, and concatenate the data points for all sequences. This produces a single set of training data and a single set of test data, which are not divided into separate sequences.  

In [ ]:
X_tr = np.concatenate(X_by_seq_tr, axis=0)  # combine features into one matrix -- use this as input features for training a classifier
y_tr = np.concatenate(y_by_seq_tr)  # combine target labels into one list -- use this as target labels for training a classifier

In [ ]:
X_test = np.concatenate(X_by_seq_test, axis=0)  # combine features into one matrix -- use this as input features for testing a classifier
y_test = np.concatenate(y_by_seq_test)  # combine target labels into one list -- use this as target labels for evaluating a classifier